In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/final_expression(proteomics).csv',index_col=0)
train.head()

,cell_id,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,NIPAL3,LAS1L,ENPP4,SEMA3F,CFTR,ANKIB1,KRIT1,RAD52,BAD,LAP3,CD99,HS3ST1,ABP1,WNT16,HECW1,MAD1L1,LASP1,SNX11,TMEM176A,M6PR,KLHL13,CYP26B1,ICA1,DBNDD1,ALS2,CASP10,CFLAR,TFPI,...,MOCS1P1,Unnamed: 17699,Unnamed: 17700,Unnamed: 17701,Unnamed: 17702,Unnamed: 17703,Unnamed: 17704,Unnamed: 17705,Unnamed: 17706,Unnamed: 17707,Unnamed: 17708,Unnamed: 17709,Unnamed: 17710,GOLGA6L7P,Unnamed: 17712,HPR,Unnamed: 17714,Unnamed: 17715,Unnamed: 17716,MIA,Unnamed: 17718,LINC00514,Unnamed: 17720,OR1D5,ZNF234,Unnamed: 17723,Unnamed: 17724,Unnamed: 17725,Unnamed: 17726,MYH4,LINC00526,PPY2,Unnamed: 17730,Unnamed: 17731,KRT18P55,Unnamed: 17733,POLRMTP1,UBL5P2,TBC1D3P5,Unnamed: 17737
4,AU565,6.280456,2.734091,10.332254,6.078175,3.499351,3.264717,3.234723,8.355618,6.182757,4.165327,3.498688,7.513071,4.141661,3.720063,5.846889,3.333278,9.237908,4.027031,3.589948,7.504017,6.228082,6.248527,3.058912,4.703172,2.803378,2.738361,5.464495,3.614326,4.033113,5.298511,9.477702,3.899033,2.868553,4.052416,3.649128,3.519793,2.948846,5.780068,3.608751,...,3.683118,4.431285,2.840129,2.947577,3.024620,5.252209,3.034330,3.068204,3.387589,3.337342,4.814958,3.325501,7.008736,3.972657,4.940006,2.672492,2.935951,3.067417,8.052976,3.343974,2.936925,6.264353,3.433707,3.507192,4.131605,2.758358,3.471807,4.456647,3.408669,2.544332,3.918447,3.022542,3.691863,8.939406,3.187884,3.110821,3.151326,8.864750,3.067727,10.839140
38,HCC1143,7.581938,2.814303,10.363326,3.770037,3.394502,3.111186,6.228677,8.440833,8.005206,5.669074,3.920152,6.977415,4.028448,3.825615,4.510053,3.100893,7.867586,3.314725,3.523449,7.143068,6.286205,8.171933,6.141540,3.405248,2.830928,3.036038,6.220310,4.627353,4.350937,3.219298,9.429471,5.307767,3.210161,3.823032,3.684000,3.586665,3.224011,4.790886,7.461998,...,3.343367,3.712943,2.910695,2.967051,2.915858,5.799966,3.016316,3.168884,3.242940,3.140979,3.683203,3.403862,3.438524,3.922050,3.542556,2.644228,2.709919,2.876814,8.506764,3.768990,2.775604,3.591381,3.200963,3.238828,4.059109,2.643087,3.059134,3.905421,3.524872,2.564325,4.324982,3.209396,3.122151,8.138551,3.539049,3.100517,2.984753,9.361984,3.225168,5.787956
30,HCC1187,9.013253,2.952093,9.347429,4.982836,4.122282,3.290773,3.014210,5.551352,5.032812,7.126702,3.528140,6.863849,4.088147,4.156939,4.708238,3.367269,9.644941,4.352032,3.941189,6.865550,8.647514,6.263462,3.264526,3.230709,3.009926,2.937750,5.383247,3.973603,4.579067,2.833575,9.635820,3.812246,3.247300,7.351081,4.217167,3.748515,2.976189,5.468538,3.472876,...,3.537663,3.152477,4.718031,3.040643,3.467101,5.908406,3.362304,3.264387,3.110702,3.548591,3.764868,3.429483,3.542451,4.061702,7.417070,2.904242,2.794378,3.369752,8.755399,3.973710,3.127434,7.497050,2.904973,3.350337,4.604941,3.138869,3.122248,4.148480,3.501497,2.914548,4.450711,3.301961,3.214124,8.055036,3.088815,2.938520,2.985197,9.628198,3.425641,7.251143
44,HCC1395,7.435151,2.832570,10.344827,3.877500,3.555658,3.511154,7.652886,8.245466,5.650228,6.008727,4.185011,6.575009,3.898302,4.047453,4.393314,3.045507,8.530566,4.079485,3.547123,7.806341,5.663647,9.795922,3.357268,3.927484,2.818479,3.005695,7.039383,4.069374,4.201485,3.189276,10.100934,4.159518,3.259254,3.263667,3.855615,3.638196,3.088089,4.980494,5.235316,...,3.347569,2.633453,4.818657,3.253613,3.469989,6.426684,3.372856,3.208805,3.124845,3.526297,3.832570,3.530187,3.644112,3.700271,3.477185,2.880257,2.996822,2.801816,8.598022,7.035974,2.946302,3.420246,3.794857,3.334150,3.933930,2.493943,2.737944,4.413332,3.630764,2.641512,4.761300,3.326525,3.813866,8.703719,2.891492,3.520857,2.984267,8.766315,3.487581,6.935819
29,HCC1500,6.981643,2.972959,10.482541,5.322880,3.110931,3.256572,2.999829,7.626167,5.666042,4.743610,3.152101,6.255289,3.904077,4.619068,5.142373,3.070199,8.778046,4.342619,3.653784,6.758323,5.303503,7.404338,3.114771,3.181848,2.821173,2.742834,6.799375,4.071830,5.596802,3.096277,9.740624,5.158979,2.880143,7.656816,4.486412,3.697275,2.894710,4.643450,4

In [4]:
target=train['cell_id']

In [5]:
train.drop(['cell_id'], axis=1, inplace=True)
train.head()

,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,STPG1,NIPAL3,LAS1L,ENPP4,SEMA3F,CFTR,ANKIB1,KRIT1,RAD52,BAD,LAP3,CD99,HS3ST1,ABP1,WNT16,HECW1,MAD1L1,LASP1,SNX11,TMEM176A,M6PR,KLHL13,CYP26B1,ICA1,DBNDD1,ALS2,CASP10,CFLAR,TFPI,NDUFAF7,...,MOCS1P1,Unnamed: 17699,Unnamed: 17700,Unnamed: 17701,Unnamed: 17702,Unnamed: 17703,Unnamed: 17704,Unnamed: 17705,Unnamed: 17706,Unnamed: 17707,Unnamed: 17708,Unnamed: 17709,Unnamed: 17710,GOLGA6L7P,Unnamed: 17712,HPR,Unnamed: 17714,Unnamed: 17715,Unnamed: 17716,MIA,Unnamed: 17718,LINC00514,Unnamed: 17720,OR1D5,ZNF234,Unnamed: 17723,Unnamed: 17724,Unnamed: 17725,Unnamed: 17726,MYH4,LINC00526,PPY2,Unnamed: 17730,Unnamed: 17731,KRT18P55,Unnamed: 17733,POLRMTP1,UBL5P2,TBC1D3P5,Unnamed: 17737
4,6.280456,2.734091,10.332254,6.078175,3.499351,3.264717,3.234723,8.355618,6.182757,4.165327,3.498688,7.513071,4.141661,3.720063,5.846889,3.333278,9.237908,4.027031,3.589948,7.504017,6.228082,6.248527,3.058912,4.703172,2.803378,2.738361,5.464495,3.614326,4.033113,5.298511,9.477702,3.899033,2.868553,4.052416,3.649128,3.519793,2.948846,5.780068,3.608751,5.705409,...,3.683118,4.431285,2.840129,2.947577,3.024620,5.252209,3.034330,3.068204,3.387589,3.337342,4.814958,3.325501,7.008736,3.972657,4.940006,2.672492,2.935951,3.067417,8.052976,3.343974,2.936925,6.264353,3.433707,3.507192,4.131605,2.758358,3.471807,4.456647,3.408669,2.544332,3.918447,3.022542,3.691863,8.939406,3.187884,3.110821,3.151326,8.864750,3.067727,10.839140
38,7.581938,2.814303,10.363326,3.770037,3.394502,3.111186,6.228677,8.440833,8.005206,5.669074,3.920152,6.977415,4.028448,3.825615,4.510053,3.100893,7.867586,3.314725,3.523449,7.143068,6.286205,8.171933,6.141540,3.405248,2.830928,3.036038,6.220310,4.627353,4.350937,3.219298,9.429471,5.307767,3.210161,3.823032,3.684000,3.586665,3.224011,4.790886,7.461998,7.340608,...,3.343367,3.712943,2.910695,2.967051,2.915858,5.799966,3.016316,3.168884,3.242940,3.140979,3.683203,3.403862,3.438524,3.922050,3.542556,2.644228,2.709919,2.876814,8.506764,3.768990,2.775604,3.591381,3.200963,3.238828,4.059109,2.643087,3.059134,3.905421,3.524872,2.564325,4.324982,3.209396,3.122151,8.138551,3.539049,3.100517,2.984753,9.361984,3.225168,5.787956
30,9.013253,2.952093,9.347429,4.982836,4.122282,3.290773,3.014210,5.551352,5.032812,7.126702,3.528140,6.863849,4.088147,4.156939,4.708238,3.367269,9.644941,4.352032,3.941189,6.865550,8.647514,6.263462,3.264526,3.230709,3.009926,2.937750,5.383247,3.973603,4.579067,2.833575,9.635820,3.812246,3.247300,7.351081,4.217167,3.748515,2.976189,5.468538,3.472876,5.302612,...,3.537663,3.152477,4.718031,3.040643,3.467101,5.908406,3.362304,3.264387,3.110702,3.548591,3.764868,3.429483,3.542451,4.061702,7.417070,2.904242,2.794378,3.369752,8.755399,3.973710,3.127434,7.497050,2.904973,3.350337,4.604941,3.138869,3.122248,4.148480,3.501497,2.914548,4.450711,3.301961,3.214124,8.055036,3.088815,2.938520,2.985197,9.628198,3.425641,7.251143
44,7.435151,2.832570,10.344827,3.877500,3.555658,3.511154,7.652886,8.245466,5.650228,6.008727,4.185011,6.575009,3.898302,4.047453,4.393314,3.045507,8.530566,4.079485,3.547123,7.806341,5.663647,9.795922,3.357268,3.927484,2.818479,3.005695,7.039383,4.069374,4.201485,3.189276,10.100934,4.159518,3.259254,3.263667,3.855615,3.638196,3.088089,4.980494,5.235316,4.594221,...,3.347569,2.633453,4.818657,3.253613,3.469989,6.426684,3.372856,3.208805,3.124845,3.526297,3.832570,3.530187,3.644112,3.700271,3.477185,2.880257,2.996822,2.801816,8.598022,7.035974,2.946302,3.420246,3.794857,3.334150,3.933930,2.493943,2.737944,4.413332,3.630764,2.641512,4.761300,3.326525,3.813866,8.703719,2.891492,3.520857,2.984267,8.766315,3.487581,6.935819
29,6.981643,2.972959,10.482541,5.322880,3.110931,3.256572,2.999829,7.626167,5.666042,4.743610,3.152101,6.255289,3.904077,4.619068,5.142373,3.070199,8.778046,4.342619,3.653784,6.758323,5.303503,7.404338,3.114771,3.181848,2.821173,2.742834,6.799375,4.071830,5.596802,3.096277,9.740624,5.158979,2.880143,7.656816,4.486412,3.697275,2.894710,4.643450,4.0

In [6]:
train_scaled = minmax_scale(train, axis = 0)

In [7]:
ncol = train_scaled.shape[1]
X_train, X_test, Y_train, Y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))

In [8]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
import pickle

# Deep Autoencoder

# this is the size of our encoded representations
encoding_dim = 24

# this is our input placeholder; 784 = 28 x 28
input = Input(shape = (ncol, ))

my_epochs = 50

# "encoded" is the encoded representation of the inputs
encoded = Dense(encoding_dim * 4, activation='relu')(input)
encoded = Dense(encoding_dim * 2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# "decoded" is the lossy reconstruction of the input
decoded = Dense(encoding_dim * 2, activation='relu')(encoded)
decoded = Dense(encoding_dim * 4, activation='relu')(decoded)
decoded = Dense(ncol, activation='sigmoid')(decoded)

# this model maps an input to its reconstruction
autoencoder = Model(input, decoded)

# Separate Encoder model

# this model maps an input to its encoded representation
encoder = Model(input, encoded)

# Separate Decoder model

# create a placeholder for an encoded (32-dimensional) input
encoded_input = Input(shape=(encoding_dim, ))
# retrieve the layers of the autoencoder model
decoder_layer1 = autoencoder.layers[-3]
decoder_layer2 = autoencoder.layers[-2]
decoder_layer3 = autoencoder.layers[-1]
# create the decoder model
decoder = Model(encoded_input, decoder_layer3(decoder_layer2(decoder_layer1(encoded_input))))

# Train to reconstruct MNIST digits

# configure model to use a per-pixel binary crossentropy loss, and the Adadelta optimizer
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# prepare input data
#(x_train, _), (x_test, y_test) = mnist.load_data()
x_train, x_test, y_train, y_test = train_test_split(train_scaled, target, train_size = 0.9, random_state = seed(2017))
# normalize all values between 0 and 1 and flatten the 28x28 images into vectors of size 784
# x_train = x_train.astype('float32') / 255.
# x_test = x_test.astype('float32') / 255.
# x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
# x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape)
print(x_test.shape)

# Train autoencoder for 50 epochs

autoencoder.fit(x_train, x_train, epochs=my_epochs, batch_size=8, shuffle=True, validation_data=(x_test, x_test),
                verbose=2)

# after 100 epochs the autoencoder seems to reach a stable train/test lost value

# Visualize the reconstructed encoded representations

# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)


encoded_train = pd.DataFrame(encoder.predict(train_scaled))
encoded_train = encoded_train.add_prefix('feature_')
encoded_train.head()

K.clear_session()

(10, 17737)
(2, 17737)
Epoch 1/50
2/2 - 1s - loss: 0.6932 - val_loss: 0.6932 - 890ms/epoch - 445ms/step
Epoch 2/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 66ms/epoch - 33ms/step
Epoch 3/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 54ms/epoch - 27ms/step
Epoch 4/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 55ms/epoch - 27ms/step
Epoch 5/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 60ms/epoch - 30ms/step
Epoch 6/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 58ms/epoch - 29ms/step
Epoch 7/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 58ms/epoch - 29ms/step
Epoch 8/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 51ms/epoch - 26ms/step
Epoch 9/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 54ms/epoch - 27ms/step
Epoch 10/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 51ms/epoch - 25ms/step
Epoch 11/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 53ms/epoch - 26ms/step
Epoch 12/50
2/2 - 0s - loss: 0.6932 - val_loss: 0.6932 - 55ms/epoch - 28ms/step
Epoch 13/50
2/2 - 0s - l

In [9]:
encoded_train

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23
0,0.000000,0.710394,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.545664,0.0,0.085801,0.000000,0.0,1.336402,0.384187,0.348203,0.0,0.355773,0.000000,0.0,0.428036,0.000000,0.00000
1,0.000000,0.696376,0.000000,0.0,0.0,0.0,0.0,0.0,0.010123,0.799780,0.0,0.434972,0.000000,0.0,1.149010,0.586554,0.049011,0.0,0.605048,0.257021,0.0,0.489265,0.000000,0.00000
2,0.000000,1.215775,0.000000,0.0,0.0,0.0,0.0,0.0,0.772779,0.794977,0.0,0.626683,0.000000,0.0,1.577695,0.897371,0.465376,0.0,0.750446,1.049645,0.0,0.307589,0.000000,0.08214
3,0.000000,0.682472,0.000000,0.0,0.0,0.0,0.0,0.0,0.516826,0.258262,0.0,0.661176,0.000000,0.0,1.021875,0.948769,0.000000,0.0,0.286125,1.371942,0.0,0.246807,0.177881,0.30059
4,0.000000,0.480889,0.000000,0.0,0.0,0.0,0.0,0.0,0.250851,0.562255,0.0,0.310229,0.629506,0.0,1.080855,0.743001,0.031574,0.0,0.234195,0.610889,0.0,0.159116,0.000000,0.00000
5,0.000000,0.136141,0.000000,0.0,0.0,0.0,0.0,0.0,0.550823,0.282176,0.0,0.063890,0.000000,0.0,0.899803,0.797713,0.018595,0.0,0.000000,0.627384,0.0,0.000000,0.000000,0.00000
6,0.000000,0.301715,0.105603,0.0,0.0,0.0,0.0,0.0,0.000000,0.683231,0.0,0.311638,0.144535,0.0,1.259411,0.542596,0.121301,0.0,0.285597,0.632687,0.0,0.549906,0.000000,0.00764
7,0.000000,0.421859,0.392142,0.0,0.0,0.0,0.0,0.0,0.000000,0.851476,0.0,0.102579,0.124591,0.0,2.065220,0.918980,0.691285,0.0,0.710758,0.000000,0.0,0.554741,0.000000,0.00000
8,0.081184,0.848633,0.000000,0.0,0.0,0.0,0.0,0.0,0.173755,0.307862,0.0,0.161394,0.000000,0.0,1.023499,0.126199,0.000000,0.0,0.000000,0.775267,0.0,0.379179,0.000000,0.00000
9,0.000000,0.249305,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.663164,0.0,0.036240,0.000000,0.0,1.304386,0.510786,0.000000,0.0,0.325717,0.489773,0.0,0.448704,0.000000,0.00000


In [10]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 17737)]           0         
                                                                 
 dense (Dense)               (None, 96)                1702848   
                                                                 
 dense_1 (Dense)             (None, 48)                4656      
                                                                 
 dense_2 (Dense)             (None, 24)                1176      
                                                                 
 dense_3 (Dense)             (None, 48)                1200      
                                                                 
 dense_4 (Dense)             (None, 96)                4704      
                                                                 
 dense_5 (Dense)             (None, 17737)             172048

In [11]:
encoded_train = encoded_train.loc[:, (encoded_train != 0).any(axis=0)]
encoded_train = encoded_train.drop(columns=encoded_train.columns[encoded_train.eq(0).mean() > 0.8])
encoded_train.shape

(12, 12)

In [12]:
encoded_train.to_excel('/content/drive/MyDrive/Datasets/dae_expression(proteomics).xlsx', index=False)